In [24]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def load_dataset(dataset_dir, image_size=(64, 64)):
    processed_images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    class_map = {class_name: i for i, class_name in enumerate(class_names)}
    
    for class_name in class_names:
        class_folder = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_folder):
            class_images = []
            for filename in os.listdir(class_folder):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(class_folder, filename)
                    image = cv2.imread(image_path)
                    if image is not None:
                        image = cv2.resize(image, image_size)
                        class_images.append(image)
                        labels.append(class_map[class_name])
            if class_images:
                processed_images.extend(class_images)
        else:
            print(f"Warning: '{class_folder}' is not a valid directory.")
    
    if not processed_images:
        raise ValueError("No valid images found in the dataset.")
    
    processed_images = np.array(processed_images)
    labels = np.array(labels)
    
    return processed_images, labels, class_map

    return processed_images, labels, class_map

# Define the path to your dataset directory
dataset_directory = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Train"

# Load and preprocess the dataset
images, labels, class_map = load_dataset(dataset_directory)

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [37]:
from tensorflow.keras import layers, models, Input
import tensorflow as tf

# Define a custom RNN cell to simulate continuous dynamics
class ContinuousTimeRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(ContinuousTimeRNNCell, self).__init__(**kwargs)
        self.units = units
        self.state_size = units  # Define state_size attribute

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer='glorot_uniform', name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units), initializer='orthogonal', name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units,), initializer='zeros', name='bias')
        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]
        h = tf.matmul(inputs, self.kernel) + tf.matmul(prev_output, self.recurrent_kernel) + self.bias
        output = tf.nn.relu(h)
        return output, [output]

    def get_initial_state(self, inputs=None, batch_size=32, dtype=tf.float32):
        return [tf.zeros((batch_size, self.units), dtype=dtype)]


def build_lnn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.RepeatVector(10)(x)  # Repeat vector to create a sequence
    # Instantiate the custom RNN cell
    rnn_cell = ContinuousTimeRNNCell(64)
    # Create the RNN layer with the custom cell
    x = layers.RNN(rnn_cell, return_sequences=False)(x)  # Simulate the dynamic reservoir
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [38]:
# Define input shape and number of classes
input_shape = train_images[0].shape
num_classes = len(np.unique(train_labels))

# Build the model
model = build_lnn_model(input_shape, num_classes)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

# Evaluate the model
# test_loss, test_accuracy = model.evaluate(val_images, val_labels)
# print(f"Test Loss: {test_loss}")
# print(f"Test Accuracy: {test_accuracy}")

# # Optionally, save the model
# model.save("lnn_model.h5")


Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 88ms/step - accuracy: 0.1561 - loss: 54.7108 - val_accuracy: 0.3214 - val_loss: 1.8631
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.4101 - loss: 1.7768 - val_accuracy: 0.4107 - val_loss: 1.6877
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.4692 - loss: 1.6124 - val_accuracy: 0.4420 - val_loss: 1.7504
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.5176 - loss: 1.4201 - val_accuracy: 0.4263 - val_loss: 1.6248
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.5298 - loss: 1.3112 - val_accuracy: 0.4754 - val_loss: 1.6531
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - accuracy: 0.5909 - loss: 1.1674 - val_accuracy: 0.4509 - val_loss: 1.7433
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.6136 - loss: 1.0966 - val_accuracy: 0.4710 - val_loss: 1.5835
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - accuracy: 0.6588 - loss: 1.0528 - val_accuracy: 0.4621 -

In [39]:
import cv2
import numpy as np

def preprocess_image(image_path, image_size=(64, 64)):
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to load image from '{image_path}'.")
        return None
    
    # Resize the image to the specified size
    image = cv2.resize(image, image_size)
    
    # Normalize pixel values (optional, depends on model input)
    image = image / 255.0  # Assuming pixel values range from 0 to 255
    
    # Add batch dimension to match model input shape
    image = np.expand_dims(image, axis=0)
    
    return image

def predict_image(model, image_path, class_map):
    # Preprocess the image
    image = preprocess_image(image_path)
    if image is None:
        return None
    
    # Make predictions using the model
    predictions = model.predict(image)
    
    # Get the predicted class index
    predicted_class_index = np.argmax(predictions[0])
    
    # Get the predicted class name from the class map
    predicted_class_name = [k for k, v in class_map.items() if v == predicted_class_index][0]
    
    return predicted_class_name

# Example usage
image_path = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Train\actinic keratosis\ISIC_0025780.jpg" # Specify the path to your image file
predicted_class = predict_image(model, image_path, class_map)

if predicted_class is not None:
    print(f"Predicted class': {predicted_class}")

test_loss, test_accuracy = model.evaluate(val_images, val_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Optionally, save the model
model.save("lnn_model.h5")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step
Predicted class': pigmented benign keratosis
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4055 - loss: 1.9710


Test Loss: 1.859963059425354
Test Accuracy: 0.453125
